In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader
import timm

import numpy as np
import pandas as pd

import gc
import os
from glob import glob 
from tqdm.notebook import tqdm

from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier

from src.dataset import DamageDataset, OrdinalBinDamageDataset
from src.agents import EmbedingsAgent, InferenceAgent
from src.utils import estimate_maximum_batch_size

## Embedings

In [2]:
torch.cuda.empty_cache()
gc.collect()

net = timm.create_model("efficientnet_b0", pretrained=True, num_classes=11)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

agent = EmbedingsAgent(net, device)
agent.load_weights('exp/stellar-frost-8/BEST.PTH')
agent.truncate_model()

cuda
Weights are loaded from exp/stellar-frost-8/BEST.PTH


In [3]:
shape = (3, 512, 512)
batch_size = estimate_maximum_batch_size(agent.model, device, shape)
print('batch_size :', batch_size)

root = 'data/train'
split = 'data/splits/train.csv'
metadata = 'data/train_meta.csv'

dataset = DamageDataset(root, split, metadata)
loader = DataLoader(dataset, batch_size=batch_size, num_workers=4)

batch_size : 128


In [4]:
df_embedings = pd.DataFrame(columns=['ID'])
df_embedings.set_index('ID', inplace=True)

In [7]:
for idx, features, _ in tqdm(loader):
    embedings = agent.get_embedings(features)
    embedings = np.hsplit(embedings, embedings.shape[1])
    
    data = {f'embeding_{i}' : embeding[:,0] for i, embeding in enumerate(embedings)}
    data['ID'] = idx
    df = pd.DataFrame(data=data)
    df.set_index('ID', inplace=True)
    
    df_embedings = pd.concat([df_embedings, df])

  0%|          | 0/153 [00:00<?, ?it/s]

In [15]:
df_embedings.to_csv('data/embedings/train.csv', index='ID')

In [16]:
root = 'data/train'
split = 'data/splits/valid.csv'
metadata = 'data/train_meta.csv'

dataset = DamageDataset(root, split, metadata)
loader = DataLoader(dataset, batch_size=batch_size, num_workers=4)

In [17]:
df_embedings = pd.DataFrame(columns=['ID'])
df_embedings.set_index('ID', inplace=True)

In [18]:
for idx, features, _ in tqdm(loader):
    embedings = agent.get_embedings(features)
    embedings = np.hsplit(embedings, embedings.shape[1])
    
    data = {f'embeding_{i}' : embeding[:,0] for i, embeding in enumerate(embedings)}
    data['ID'] = idx
    df = pd.DataFrame(data=data)
    df.set_index('ID', inplace=True)
    
    df_embedings = pd.concat([df_embedings, df])

  0%|          | 0/51 [00:00<?, ?it/s]

In [19]:
df_embedings.to_csv('data/embedings/valid.csv', index='ID')

In [20]:
root = 'data/test'
split = 'data/splits/test.csv'
metadata = 'data/test_meta.csv'

dataset = DamageDataset(root, split, metadata)
loader = DataLoader(dataset, batch_size=batch_size, num_workers=4)

In [21]:
df_embedings = pd.DataFrame(columns=['ID'])
df_embedings.set_index('ID', inplace=True)

In [22]:
for idx, features, _ in tqdm(loader):
    embedings = agent.get_embedings(features)
    embedings = np.hsplit(embedings, embedings.shape[1])
    
    data = {f'embeding_{i}' : embeding[:,0] for i, embeding in enumerate(embedings)}
    data['ID'] = idx
    df = pd.DataFrame(data=data)
    df.set_index('ID', inplace=True)
    
    df_embedings = pd.concat([df_embedings, df])

  0%|          | 0/68 [00:00<?, ?it/s]

In [23]:
df_embedings.to_csv('data/embedings/test.csv', index='ID')

## Cat. features encoding

In [2]:
meta = pd.read_csv('data/train_meta.csv', index_col='ID')
train_split = pd.read_csv('data/splits/train.csv', index_col='ID')
valid_split = pd.read_csv('data/splits/valid.csv', index_col='ID')

In [3]:
cat_columns = ['growth_stage', 'damage', 'season']

In [4]:
for column in cat_columns:
    print(f'{column} : {pd.unique(meta[column])}')

growth_stage : ['S' 'V' 'M' 'F']
damage : ['WD' 'G' 'DR' 'ND' 'DS' 'PS' 'WN' 'FD']
season : ['SR2020' 'SR2021' 'LR2020' 'LR2021']


In [5]:
train_features = meta[meta.index.isin(train_split.index)][cat_columns]
valid_features = meta[meta.index.isin(valid_split.index)][cat_columns]

encoder = OneHotEncoder(sparse_output=False)
train_features = encoder.fit_transform(train_features)
valid_features = encoder.transform(valid_features)

In [19]:
data = {key : np.hsplit(train_features, train_features.shape[1])[i][:,0] for i, key in enumerate(encoder.get_feature_names_out())}
data['ID'] = train_split.index
df = pd.DataFrame(data=data)
df.set_index('ID', inplace=True)
df.to_csv('data/cat_features/train.csv', index='ID')

In [20]:
data = {key : np.hsplit(valid_features, valid_features.shape[1])[i][:,0] for i, key in enumerate(encoder.get_feature_names_out())}
data['ID'] = valid_split.index
df = pd.DataFrame(data=data)
df.set_index('ID', inplace=True)
df.to_csv('data/cat_features/valid.csv', index='ID')

## Boostings

In [2]:
train_embedings = pd.read_csv('data/embedings/train.csv', index_col='ID')
train_cat_features = pd.read_csv('data/cat_features/train.csv', index_col='ID')

valid_embedings = pd.read_csv('data/embedings/valid.csv', index_col='ID')
valid_cat_features = pd.read_csv('data/cat_features/valid.csv', index_col='ID')

train_features = train_embedings.join([train_cat_features])
valid_features = valid_embedings.join([valid_cat_features])

features_cols = train_features.columns

In [3]:
train_split = pd.read_csv('data/splits/train.csv', index_col='ID')
valid_split = pd.read_csv('data/splits/valid.csv', index_col='ID')

meta = pd.read_csv('data/train_meta.csv', index_col='ID')

In [4]:
train_targets = meta[meta.index.isin(train_split.index)].extent
train_targets = train_targets // 10
valid_targets = meta[meta.index.isin(valid_split.index)].extent
valid_targets = valid_targets // 10

target_col = 'extent'

In [5]:
train = train_features.join([train_targets])
valid = valid_features.join([valid_targets])

In [6]:
clf = XGBClassifier(n_estimators=100, max_depth=5, n_jobs=4, verbosity=2)
clf.fit(train[features_cols], train[target_col])

[01:58:10] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[01:58:11] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=5
[01:58:12] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=5
[01:58:12] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[01:58:13] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 50 extra nodes, 0 pruned nodes, max_depth=5
[01:58:14] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=5
[01:58:15] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[01:58:15] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=5
[01:58:16] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 54 extra nodes, 0 pruned nodes, max_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=4, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [7]:
preds = clf.predict(valid[features_cols])

## Split over season validation

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
lr2020_model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=8)
lr2020_agent = InferenceAgent(lr2020_model, device)
lr2020_agent.load_weights('exp/autumn-haze-44/BEST.PTH')

lr2021_model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=8)
lr2021_agent = InferenceAgent(lr2021_model, device)
lr2021_agent.load_weights('exp/misunderstood-blaze-49/BEST.PTH')

sr2020_model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=9)
sr2020_agent = InferenceAgent(sr2020_model, device)
sr2020_agent.load_weights('exp/worthy-lake-40/BEST.PTH')

sr2021_model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=9)
sr2021_agent = InferenceAgent(sr2021_model, device)
sr2021_agent.load_weights('exp/worthy-lake-40/BEST.PTH')

Weights are loaded from exp/autumn-haze-44/BEST.PTH
Weights are loaded from exp/misunderstood-blaze-49/BEST.PTH
Weights are loaded from exp/worthy-lake-40/BEST.PTH
Weights are loaded from exp/worthy-lake-40/BEST.PTH


In [38]:
agents = {
    'lr2020' : lr2020_agent,
    'lr2021' : lr2021_agent,
    'sr2020' : sr2020_agent,
    'sr2021' : sr2021_agent
}

thresholds = {
    'lr2020' : 0.6,
    'lr2021' : 0.45,
    'sr2020' : 0.6,
    'sr2021' : 0.6
}    

In [39]:
shape = (3, 512, 512)

root = 'data/train'
metadata = 'data/train_meta.csv'
meta = pd.read_csv(metadata, index_col='ID')

In [40]:
prediction_df = pd.DataFrame(columns=['ID', 'extent'])
prediction_df.set_index('ID', inplace=True)

target_df = pd.DataFrame(columns=['ID', 'target'])
target_df.set_index('ID', inplace=True)

In [41]:
for season in tqdm(agents.keys()):
    torch.cuda.empty_cache()
    gc.collect()
    
    batch_size = estimate_maximum_batch_size(agents[season].model, device, shape)
    
    split = f'data/splits/new/{season}_valid.csv'
    dataset = DamageDataset(root, split, metadata)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=4)

    prediction_df_part = pd.DataFrame(columns=['ID', 'extent'])
    prediction_df_part.set_index('ID', inplace=True)

    for idx, features, _ in tqdm(loader):
        output = agents[season].predict(features)
        output = nn.Sigmoid()(output)
        prediction = output > thresholds[season]
        prediction = prediction.to(torch.int64).cpu()
        prediction = torch.tensor([torch.where(row == 1)[0].max() + 1 if row.sum() > 0 else 0 for row in prediction])
        prediction += 1
        
        df = pd.DataFrame(data={'ID' : idx, 'extent' : prediction})
        df.set_index('ID', inplace=True)
        
        prediction_df_part = pd.concat([prediction_df_part, df])

    target_df_part = pd.DataFrame(data={
        'ID' : prediction_df_part.index, 
        'target' :  meta.loc[prediction_df_part.index].extent 
    })
    target_df_part.set_index('ID', inplace=True)

    prediction_df = pd.concat([prediction_df, prediction_df_part])
    target_df =  pd.concat([target_df, target_df_part])

prediction_df *= 10

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [42]:
np.sqrt(np.power(target_df.target - prediction_df.extent, 2).sum() / prediction_df.shape[0])

14.654016203081314

## Grads exp

In [67]:
n_classes = 11
indices = torch.arange(11)

kernel = torch.distributions.normal.Normal(torch.tensor(0.0), torch.tensor(1.0)).cdf
classes_prior_proba = torch.ones(n_classes)
costs_proportional = torch.abs(indices.unsqueeze(1) - indices)
cutpoints = torch.arange(1, n_classes+1) - 0.5

In [85]:
class BayesianLoss(nn.Module):
    def __init__(self,
                 classes_prior_proba,
                 costs_proportional,
                 kernel,
                 n_classes):
        
        super(BayesianLoss, self).__init__()

        self.Pi = classes_prior_proba
        self.C = costs_proportional
        self.K = kernel
        self.M = n_classes

    def forward(self, inputs, targets, cutpoints):
        self.C = self.C.to(inputs.device)
        self.Pi = self.Pi.to(inputs.device)

        # Create a mask for each class
        class_masks = [targets == t for t in range(self.M)]
        class_losses = []

        for t in range(self.M):
            S_t = torch.where(class_masks[t])[0]
            C_t = self.C[:, t]

            # Calculate the difference between cost values
            cost_diff = C_t[:-1] - C_t[1:]

            # Calculate the difference between cutpoints and inputs
            cutpoint_diff = cutpoints - inputs[S_t].unsqueeze(1)

            # Calculate the loss for class t
            class_loss = torch.sum(cost_diff * self.K(cutpoint_diff), dim=1) + C_t[-1]

            # Add the class loss weighted by prior probability
            class_losses.append(torch.sum(class_loss) * self.Pi[t])

        return sum(class_losses)

In [81]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = nn.Linear(in_features=1, out_features=1)

    def forward(self, x):
        return self.linear(x)

In [64]:
import itertools

In [94]:
model = Model()

n_classes = 11
indices = torch.arange(11)

kernel = torch.distributions.normal.Normal(torch.tensor(0.0), torch.tensor(1.0)).cdf
classes_prior_proba = torch.ones(n_classes)
costs_proportional = torch.abs(indices.unsqueeze(1) - indices)

cutpoints = torch.arange(10).to(torch.float32)
# cutpoints = torch.zeros(n_classes - 1)
# cutpoints.requires_grad=True
cutpoints = nn.Parameter(cutpoints)

crtiterion = BayesianLoss(classes_prior_proba, costs_proportional, kernel, n_classes)
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), (cutpoints, )), lr=0.1)
# itertools.chain(model.parameters(), (cutpoints, )
inputs = torch.rand((100, 1))
targets = torch.randint(n_classes, (100,))

In [95]:
for i in range(10):
    # inputs = torch.rand((100, 1))
    loss = crtiterion(model(inputs).flatten(), targets, cutpoints)
    print(cutpoints, loss)
    # print(loss.requires_grad)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Parameter containing:
tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.], requires_grad=True) tensor(481.9198, grad_fn=<AddBackward0>)
Parameter containing:
tensor([-0.1000,  0.9000,  1.9000,  2.9000,  3.9000,  5.0992,  6.0603,  7.0003,
         8.0000,  9.0000], requires_grad=True) tensor(472.1060, grad_fn=<AddBackward0>)
Parameter containing:
tensor([-0.2001,  0.8004,  1.8019,  2.8041,  3.8075,  5.1983,  6.1253,  7.0009,
         8.0000,  9.0000], requires_grad=True) tensor(460.8179, grad_fn=<AddBackward0>)
Parameter containing:
tensor([-0.3004,  0.7010,  1.7059,  2.7125,  3.7214,  5.2971,  6.1947,  7.0019,
         8.0000,  9.0000], requires_grad=True) tensor(448.3619, grad_fn=<AddBackward0>)
Parameter containing:
tensor([-0.4006,  0.6015,  1.6113,  2.6240,  3.6397,  5.3954,  6.2682,  7.0038,
         8.0000,  9.0000], requires_grad=True) tensor(435.1415, grad_fn=<AddBackward0>)
Parameter containing:
tensor([-0.5002,  0.5016,  1.5176,  2.5375,  3.5607,  5.4924,  6.3452,  7.0078,
      

In [25]:
list(model.children())[0].weight

Parameter containing:
tensor([[0.4021]], requires_grad=True)

In [69]:
print(dict(crtiterion.named_parameters()))
print(list(crtiterion.parameters()))

{'cutpoints': Parameter containing:
tensor([0.0500, 0.1500, 0.2500, 0.3500, 0.4500, 0.5500, 0.6500, 0.7500, 0.8500,
        0.9500], requires_grad=True)}
[Parameter containing:
tensor([0.0500, 0.1500, 0.2500, 0.3500, 0.4500, 0.5500, 0.6500, 0.7500, 0.8500,
        0.9500], requires_grad=True)]


In [70]:
optimizer.param_groups[0]['params']

[Parameter containing:
 tensor([0.0500, 0.1500, 0.2500, 0.3500, 0.4500, 0.5500, 0.6500, 0.7500, 0.8500,
         0.9500], requires_grad=True)]

In [79]:
loss.grad_fn

In [118]:
outputs = torch.squeeze(model(inputs), dim=1)

In [119]:
def get_intervals(input_tensor, cutpoints):
    # Sort the cutpoints and input tensor
    sorted_cutpoints = torch.sort(cutpoints).values
    sorted_input = torch.sort(input_tensor).values

    # Find the indices where the input_tensor crosses the cutpoints
    interval_indices = torch.searchsorted(sorted_cutpoints, sorted_input)

    return interval_indices

predictions = get_intervals(outputs, cutpoints)

In [120]:
predictions.shape

torch.Size([100])

In [121]:
cutpoints.shape

torch.Size([10])

In [122]:
def get_confusion_matrix(targets, predictions, n_classes):
    confusion_matrix = torch.zeros(n_classes, n_classes, device=predictions.device, dtype=predictions.dtype)

    indices = targets * n_classes + predictions
    ones = torch.ones_like(indices, device=predictions.device, dtype=predictions.dtype)
    confusion_matrix.view(-1).scatter_add_(0, indices, ones)

    return confusion_matrix

In [123]:
get_confusion_matrix(targets, predictions, n_classes)

tensor([[ 0, 13,  1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 10,  1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  5,  1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  8,  1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  6,  1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 10,  2,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  6,  1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  6,  2,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  9,  1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  7,  1,  0,  0,  0,  0,  0,  0,  0,  0]])